In [ ]:
import pandas as pd
import pyodbc
from sqlalchemy import engine
from pandas import ExcelWriter

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#cnxn = pyodbc.connect('''DRIVER=SQL Server;
#                        Server=192.168.35.102;
 #                       'Database=PowerBI;
 #                       Trusted_Connection=Yes
 #                     ''')
#cursor=cnxn.cursor()


In [ ]:
query='''
                        WITH table1 as 
(
SELECT  DATEPART(WEEK, th.[TranDate]) AS WeekNum,p.[Category],p.[Class],p.[StockCode],p.[Description],p.[Status], (tl.[ImpactAmount]-tl.[LineTotalCost]) AS Profit
From [PowerBI].[dbo].[TransactionLine] tl 
LEFT JOIN [PowerBI].[dbo].[Subsidiary] s ON tl.SubsidiaryId = s.[SubsidiaryId] 
LEFT JOIN [PowerBI].[dbo].[TransactionHeader] th ON tl.[TransactionId]=th.[TransactionId]
LEFT JOIN [PowerBI].[dbo].[Product] p ON tl.[ProductId] =p.[ProductId]


WHERE th.[TranDate]>= DATEADD(MONTH, -12, GETDATE()) 
AND s.Name='Wilton Bradley Limited' 
AND p.[Class] IS NOT NULL 
AND p.[Category] IS NOT NULL 
AND  tl.[GlId] =54
AND tl.TransactionType IN ('Cash Sale', 'Credit Memo','Invoice')

)

SELECT  [WeekNum],[Category],[Class],[StockCode],[Description],[Status], SUM(Profit) AS TotalProfit
FROM table1  
GROUP BY [StockCode], [WeekNum], [Description],[Category],[Class],[Status]
ORDER BY WeekNum ASC, TotalProfit DESC

               '''



In [ ]:
#df=pd.read_sql(query, cnxn)
#cursor.close()
#df

In [ ]:
engine=engine.create_engine('mssql://@192.168.35.102/PowerBi?driver=SQL Server')

In [ ]:
df=pd.read_sql_query(query, engine)
df

In [ ]:
df.groupby(["Class"])["TotalProfit"].sum()

In [ ]:
query_inv='''
SELECT 
    p.[StockCode],
    SUM(COALESCE(i.[QuantityOnHand], 0)) AS QuantityOnHand,
    ROUND(
        CAST(CASE
                WHEN p.[OuterCtnQty] <> 0 THEN (p.[Ctncbm] / p.[OuterCtnQty])
                ELSE 0 END AS NUMERIC(18, 4)
        ), 4) AS unit_cubicmeter
FROM 
    [PowerBi].[dbo].[Product] p 
LEFT JOIN 
    [PowerBi].[dbo].[ItemInventory] i ON p.[ProductId] = i.[ProductId]
GROUP BY 
    p.[StockCode], 
    p.[OuterCtnQty], 
    p.[Ctncbm]  -- Include only the columns used in the SELECT clause or aggregated
ORDER BY 
    [QuantityOnHand] DESC
'''

In [ ]:
df_inv=pd.read_sql_query(query_inv,engine)
df_inv["unit_cubicmeter"]=df_inv["unit_cubicmeter"].fillna(0)
df_inv["Vol"]=df_inv["QuantityOnHand"]*df_inv["unit_cubicmeter"]
df_inv

In [ ]:
def xyz(x):
    if abs(x)<2:
        return "X"
    elif abs(x)<4:
        return "Y"
    else:
        return "Z"
    
    
    
def abc(x):
    if x<80:
        return "A"
    elif x<90:
        return "B"
    else:
        return "C"

In [ ]:
def xyz1(x2):
    x1=x2.pivot_table(index=["Category", "Class", "StockCode", "Description"], 
                  columns="WeekNum", values="TotalProfit", fill_value=0).reset_index()
    x1["Average"]=x1.iloc[:,4:].mean(axis=1,skipna=True)
    x1["COV"]=x1.iloc[:,4:].std(axis=1, skipna=True)/x1["Average"]
    
   
    sns.displot(x1["COV"], kde=True, bins=50)

    plt.show()
    
    x1["XYZ"]=x1["COV"].apply(xyz)
    

    
    return x1

In [ ]:
def abc1(a):
    a1=a.groupby(["Category", "Class", "StockCode", "Description","Status"]).agg(TotalProfit=("TotalProfit", "sum")).sort_values(by="TotalProfit", ascending=False).reset_index()
    a1["CumProfit"]=a1["TotalProfit"].cumsum()
    a1["Profit_all"]=a1["TotalProfit"].sum()
    a1["%Profit"]=(a1["CumProfit"]/a1["Profit_all"])*100
    a1["Bottom CumProfit"]=a1["Profit_all"]-a1["CumProfit"]
    a1["ABC"]=a1["%Profit"].apply(abc)
    
    return a1

In [ ]:
def merge(x,a):
    #Only choosing [0, 1, 2, 3, 55, 56, 57] columns for merging
    select_x=list(range(0,4))+list(range(57,x.shape[1]))
    #select_a=[2]+list(range(4,a.shape[1]))
    select_a=list(range(2,3))+list(range(4,a.shape[1]))
    
    
    m = pd.merge(x.iloc[:, select_x], a.iloc[:,select_a], how="inner", on="StockCode").sort_values(by="%Profit")
    m1=pd.merge(m, df_inv, how="left", on='StockCode').sort_values(by="%Profit")
    return m1

In [ ]:
writer=pd.ExcelWriter("S:/sales office\Vishnu\High inventory products/ABC & XYZ/ABC&XYZ_Profit.xlsx")


# OSPREY

In [ ]:
osprey=df[df["Class"]=="Osprey"]
x=xyz1(osprey)
x

In [ ]:
osprey


In [ ]:
a=abc1(osprey)
a

In [ ]:
m_osprey=merge(x,a)
m_osprey

# BESTWAY

In [ ]:
bestway=df[df["Class"]=="Bestway"]
x=xyz1(bestway)
x

In [ ]:
a=abc1(bestway)
a

In [ ]:
m_bestway=merge(x,a)
m_bestway

# CLEARWATER

In [ ]:
clearwater=df[df["Class"]=="Clearwater"]
x=xyz1(clearwater)
x

In [ ]:
a=abc1(clearwater)
a

In [ ]:
m_clearwater=merge(x,a)
m_clearwater

# TOYRIFFIC

In [ ]:
toyrific=df[df["Class"]=="Toyrific"]
x=xyz1(toyrific)
x

In [ ]:
a=abc1(toyrific)
a

In [ ]:
m_toyrific=merge(x,a)
m_toyrific

# Yello

In [ ]:
yello=df[df["Class"]=="Yello"]
x=xyz1(yello)
x

In [ ]:
a=abc1(yello)
a

In [ ]:
m_yello=merge(x,a)
m_yello

# VW

In [ ]:
vw=df[df["Class"]=="VW"]
x=xyz1(vw)
x

In [ ]:
a=abc1(vw)
a

In [ ]:
m_vw=merge(x,a)
m_vw

# Urban Beach

In [ ]:
urban=df[df["Class"]=="Urban Beach"]
x=xyz1(urban)
x

In [ ]:
a=abc1(urban)
a
a.head()

In [ ]:
select_x=list(range(0,4))+list(range(55,x.shape[1]))
select_a=list(range(2,3))+list(range(4,a.shape[1]))
m_urban=pd.merge(x.iloc[:,select_x], a.iloc[:,select_a], how="inner", on="StockCode").sort_values(by="%Profit")
m_urban


# Xootz

In [ ]:
xootz=df[df["Class"]=="Xootz"]
x=xyz1(xootz)
x

In [ ]:
a=abc1(xootz)
a

In [ ]:
m_xootz=merge(x,a)
m_xootz

# Lay-Z-Spa

In [ ]:
layz=df[df["Class"]=="Lay-Z-Spa"]
x=xyz1(layz)
x

In [ ]:
a=abc1(layz)
a

In [ ]:
m_layz=merge(x,a)
m_layz

In [ ]:
path="S:/sales office\Vishnu\High inventory products/ABC & XYZ/ABC&XYZ_Profit.xlsx"

In [ ]:
with pd.ExcelWriter(path) as writer:
    m_osprey.to_excel(writer, "Osprey", index=False)
    m_bestway.to_excel(writer, "Bestway", index=False)
    m_clearwater.to_excel(writer, "Clearwater", index=False)
    m_toyrific.to_excel(writer, "Toyrific", index=False)
    m_yello.to_excel(writer, "yello", index=False)
    m_urban.to_excel(writer, "Urban Beach", index=False)
    m_layz.to_excel(writer, "Lay-Z-Spa", index=False)
    m_xootz.to_excel(writer, "Xootz", index=False)
    m_vw.to_excel(writer, "VW", index=False)